# Simple Injector Examples
This example provides injecting way with non-nested some classes and some variables.
The class following a protocol

## load packages

In [1]:
from dataclasses import dataclass
from datetime import datetime
from typing import Protocol

from injector import Module, Binder, inject, Injector, provider, singleton

## Setting up Classes

In [2]:
class Transportation(Protocol):
    """Protocol to be followed transportation
    """
    def transport(self) -> None:
        ...

    def __repr__(self) -> str:
        ...


class Ship:
    """A kind of transporation
    """
    def __init__(self, name: str = 'titanic') -> None:
        self.name = name

    def transport(self) -> None:
        print(f'{self.name} goes on the sea')

    def __repr__(self) -> str:
        return self.name


@inject
@dataclass
class Route:
    """Represents a route by ship
    """
    departure_time: datetime
    ship: Transportation

    def __repr__(self) -> str:
        return f"""
            {self.departure_time=}
            {self.ship=}
        """


## Injection

### With Fixed Configuration
This module provides a route with current departure

In [3]:
class CurrentRouteModule(Module):
    """Creates `Routes` from current time
    """
    def configure(self, binder: Binder) -> None:
        binder.bind(Transportation, to=Ship)
        binder.bind(datetime, to=datetime.now())


injector1 = Injector(CurrentRouteModule)
route1 = injector1.get(Route)
print(route1)


            self.departure_time=datetime.datetime(2020, 5, 19, 14, 34, 24, 891598)
            self.ship=titanic
        


### With Variable Configuration
This module provides a route with departure time set by outer scope.
There is several ways to achieve this.

#### inject with a config class

In [11]:
@dataclass
class DepartureConfig:
    departure_time: datetime


class RouteModule(Module):
    """Common Route Module
    """

    def configure(self, binder: Binder) -> None:
        binder.bind(Transportation, to=Ship)

    @provider
    def provide_departure(self, config: DepartureConfig) -> datetime:
        return config.departure_time


def configure_departure_for_2020_new_year(binder: Binder) -> None:
    binder.bind(
        DepartureConfig,
        to=DepartureConfig(
            departure_time=datetime(
                year=2020, month=1, day=1,
            )
        ),
    )


def departure_config_for_2021_new_year(binder: Binder) -> None:
    binder.bind(
        DepartureConfig,
        to=DepartureConfig(
            departure_time=datetime(
                year=2020, month=1, day=1,
            )
        ),
    )

In [12]:
injector2 = Injector([configure_departure_for_2020_new_year, RouteModule()])
route2 = injector2.get(Route)
print(route2)


            self.departure_time=datetime.datetime(2020, 1, 1, 0, 0)
            self.ship=titanic
        


In [13]:
injector3 = Injector([departure_config_for_2021_new_year, RouteModule()])
route3 = injector3.get(Route)
print(route3)


            self.departure_time=datetime.datetime(2020, 1, 1, 0, 0)
            self.ship=titanic
        


Now, we can get two routes from common `RouteModule`

Now, we can get two routes from common `RouteModule`